In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
from qiskit import QuantumCircuit, Aer, execute

# PARAMETERS

# Beats per minute
bpm = 4  # /4
measures = 60
beats = bpm * measures

# Shots for key determination
key_shots = 50

# DICTIONARY FOR NOTE TYPES

# Dictionary mapping binary representations to note durations
type_conversion_dict = {
    '000': 4,    # whole
    '001': 3,    # dotted half
    '010': 2,    # half
    '011': 1.5,  # dotted quarter
    '100': 1,    # quarter
    '101': 0.5,  # eighth
    '110': 0.25, # sixteenth
    '111': 1     # quarter
}

# TYPE CIRCUIT

# Quantum circuit to determine note types
type_circuit = QuantumCircuit(3, 3)
type_circuit.h([0, 1, 2])
type_circuit.measure([0, 1, 2], [0, 1, 2])

# Execute the type circuit
t_shots_number = round(beats * 4)
backend = Aer.get_backend('qasm_simulator')
job_t = execute(type_circuit, backend, shots=t_shots_number, memory=True)
result_t = job_t.result()
type_outcomes = result_t.get_memory()

# Convert outcomes to note types
initial_types_list = []
for i in type_outcomes:
    note_type = type_conversion_dict[i]
    initial_types_list.append(note_type)

# Determine the sequence of note types within the given beats
count = 0
types_list = []
for i in initial_types_list:
    if (count + i) <= beats:
        types_list.append(i)
        count += i

# Total number of notes
number_of_notes = len(types_list)


# GET NOTES

# Quantum circuit to generate musical notes
circuit = QuantumCircuit(3, 3)
circuit.h([0, 1, 2])
circuit.measure([0, 1, 2], [0, 1, 2])

# Execute the circuit to generate notes
backend = Aer.get_backend('qasm_simulator')
job = execute(circuit, backend, shots=number_of_notes, memory=True)
result = job.result()
outcomes = result.get_memory()

# Convert outcomes to musical notes
notes_list = []
note_conversion_dict = {
    '000': 'A', 
    '001': 'B', 
    '010': 'C', 
    '011': 'D',
    '100': 'E', 
    '101': 'F', 
    '110': 'G', 
    '111': 'r'
}

for i in outcomes:
    note = note_conversion_dict[i]
    notes_list.append(note)


# GET NOTE "LEVEL"

# Quantum circuit to determine note levels
level_circ = QuantumCircuit(1, 1)
level_circ.h(0)
level_circ.measure(0, 0)

# Execute the circuit to determine note levels
job_l = execute(level_circ, backend, shots=t_shots_number, memory=True)
result_l = job_l.result()
level_outcomes = result_l.get_memory()

# Convert outcomes to note levels
levels_list = []
levels = ["'", "''"]
for i in level_outcomes:
    state = int(i)
    l = str(levels[state])
    levels_list.append(l)


# GET KEY SIGNATURE

# PART 1: QUANTITY

# Quantum circuit to determine the quantity of key signatures
key_circuit = QuantumCircuit(3, 3)
key_circuit.h([0, 1, 2])
key_circuit.measure([0, 1, 2], [0, 1, 2])

# Execute the circuit to determine the quantity of key signatures
job_k = execute(key_circuit, backend, shots=10000)
result_k = job_k.result()
counts_dict = result_k.get_counts()

# Find the most probable key signature
values = list(counts_dict.values())
keys = list(counts_dict.keys())
largest_number = 0
final_key = ""
for i in range(len(values)):
    if values[i] > largest_number:
        largest_number = values[i]
        final_key = keys[i]

# Convert binary key signature to number of sharps/flats
key_conversion_dict = {
    '000': 0, '001': 1, '010': 2, '011': 3,
    '100': 4, '101': 5, '110': 6, '111': 7
}
number_of_sf = key_conversion_dict[final_key]

# PART 2: SHARPS OR FLATS

# Quantum circuit to determine whether the key signature has sharps or flats
sf_circuit = QuantumCircuit(1, 1)
sf_circuit.h(0)
sf_circuit.measure(0, 0)

# Execute the circuit to determine sharps or flats
job_sf = execute(sf_circuit, backend, shots=10000)
result_sf = job_sf.result()
sf_dict = result_sf.get_counts()

# Determine if the key is sharp or flat based on measurement results
values2 = list(sf_dict.values())
keys2 = list(sf_dict.keys())
sharp_keys = ["C", "G", "D", "A", "E", "B", "Fis", "Cis"]
flat_keys = ["C", "F", "Bes", "Ees", "Aes", "Des", "Ges", "Ces"]
if values2[0] > values2[1]:
    key = sharp_keys[number_of_sf].lower()
else:
    key = flat_keys[number_of_sf].lower()


# APPLY ACCIDENTALS TO NOTES

# Function to apply accidentals to notes based on key signature
def apply_accidentals(is_sharp, num_accidentals, notes):
    new_notes = []
    if num_accidentals > 0:
        if is_sharp:
            sharps = ['F', 'C', 'G', 'D', 'A', 'E', 'D']
            for i in range(num_accidentals):
                new_notes.append(notes[i] + "is" if notes[i].upper() in sharps[:num_accidentals] else notes[i])
        else:
            flats = ['B', 'E', 'A', 'D', 'G', 'C', 'F']
            for i in range(num_accidentals):
                new_notes.append(notes[i] + "es" if notes[i].upper() in flats[:num_accidentals] else notes[i])
    else:
        new_notes = notes
    return new_notes

# Apply accidentals to notes based on key signature
if values[0] > values[1]:
    new_notes = apply_accidentals(True, number_of_sf, notes_list)
else:
    new_notes = apply_accidentals(False, number_of_sf, notes_list)


# CONVERT NOTE TYPES

# Convert note types to their respective notation symbols
converted_types = []
for i in types_list:
    if i == 4:
        converted_types.append("1")
    elif i == 3:
        converted_types.append("2.")
    elif i == 2:
        converted_types.append("2")
    elif i == 1.5:
        converted_types.append("4.")
    elif i == 1:
        converted_types.append("8")
    elif i == 0.25:
        converted_types.append("16")

# PRINT THE SCORE

# Printing the musical score in Lilypond syntax
print("  \key " + str(key) + " \major")
for i in range(len(converted_types)):
    note = new_notes[i].lower()
    if note == "r":
        final_line = note + converted_types[i]
    else:
        final_line = note + str(levels_list[i]) + converted_types[i]
    print(final_line)